In [1]:
import tensorflow as tf
from tensorflow.keras.datasets.cifar10 import load_data

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x): # compression 2x2 -> 1x1
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

w_conv1 = weight_variable([5,5,3,64])
b_conv1 = bias_variable([64])
h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

w_conv2 = weight_variable([5, 5, 64, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

w_conv3 = weight_variable([3, 3, 64, 128])
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_pool2, w_conv3) + b_conv3)

w_conv4 = weight_variable([3, 3, 128, 128])
b_conv4 = bias_variable([128])
h_conv4 = tf.nn.relu(conv2d(h_conv3, w_conv4) + b_conv4)

w_conv5 = weight_variable([3, 3, 128, 128])
b_conv5 = bias_variable([128])
h_conv5 = tf.nn.relu(conv2d(h_conv4, w_conv5) + b_conv5)

w_fc1 = weight_variable([8 * 8 * 128, 384])
b_fc1 = bias_variable([384])

h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, w_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


w_fc2 = weight_variable([384, 10])
b_fc2 = bias_variable([10])

logits = tf.matmul(h_fc1_drop,w_fc2) + b_fc2
y_conv = tf.nn.softmax(logits)
    
#y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

#cross_entropy = -tf.reduce_sum(y*tf.log(tf.clip_by_value(y_conv,1e-10,1.0)))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
import numpy as np
def next_batch(num, data, labels):
  '''
  `num` 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴합니다.
  '''
  idx = np.arange(0 , len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_shuffle = [data[ i] for i in idx]
  labels_shuffle = [labels[ i] for i in idx]

  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [36]:
with tf.Session() as sess:
    y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
    batch = next_batch(10, x_train, sess.run(y_train_one_hot))
    print(batch[1])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
for i in  range(4001):    
    batch = next_batch(128, x_train, sess.run(y_train_one_hot))
    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
        loss_print = sess.run(loss,feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
        print("\nstep %d, training accuracy %f , loss : %f"%(i, train_accuracy, loss_print))
#        pred_y = sess.run(y_conv, feed_dict={x:batch[0], keep_prob:1.0})
#         for q in range(20):
#             print("predict : ",sess.run(tf.argmax(pred_y[q])), " , y : ",sess.run(tf.argmax(batch[1][q])))
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1], keep_prob: 0.5})        


step 0, training accuracy 0.101562 , loss : 167.148193

step 100, training accuracy 0.125000 , loss : 2.270398

step 200, training accuracy 0.132812 , loss : 2.280955

step 300, training accuracy 0.265625 , loss : 2.135610

step 400, training accuracy 0.328125 , loss : 1.805978

step 500, training accuracy 0.406250 , loss : 1.663312

step 600, training accuracy 0.367188 , loss : 1.617541

step 700, training accuracy 0.476562 , loss : 1.452972

step 800, training accuracy 0.414062 , loss : 1.581733

step 900, training accuracy 0.507812 , loss : 1.453476

step 1000, training accuracy 0.406250 , loss : 1.544656

step 1100, training accuracy 0.414062 , loss : 1.532962

step 1200, training accuracy 0.531250 , loss : 1.289706

step 1300, training accuracy 0.515625 , loss : 1.238158

step 1400, training accuracy 0.546875 , loss : 1.168049

step 1500, training accuracy 0.679688 , loss : 1.037984

step 1600, training accuracy 0.671875 , loss : 1.077959

step 1700, training accuracy 0.640625 , 